In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

# Data Preparation

In [ ]:
X_train = pd.read_csv("X_train.txt", header=None,  delim_whitespace=True)
y_train = pd.read_csv("y_train.txt", header=None, delim_whitespace=True )
X_test = pd.read_csv("X_test.txt", header=None, delim_whitespace=True )
y_test = pd.read_csv("y_test.txt", header=None, delim_whitespace=True )
subject_test = pd.read_csv("subject_test.txt", header=None, delim_whitespace=True )
subject_train = pd.read_csv("subject_train.txt", header=None, delim_whitespace=True )
features = pd.read_csv("features.txt", header=None, delim_whitespace=True )

In [ ]:
body_acc_x_train = pd.read_csv("body_acc_x_train.txt", header=None,  delim_whitespace=True)
body_acc_y_train = pd.read_csv("body_acc_y_train.txt", header=None,  delim_whitespace=True)
body_acc_z_train = pd.read_csv("body_acc_z_train.txt", header=None,  delim_whitespace=True)
body_gyro_x_train = pd.read_csv("body_gyro_x_train.txt", header=None,  delim_whitespace=True)
body_gyro_y_train = pd.read_csv("body_gyro_y_train.txt", header=None,  delim_whitespace=True)
body_gyro_z_train = pd.read_csv("body_gyro_z_train.txt", header=None,  delim_whitespace=True)
total_acc_x_train = pd.read_csv("body_gyro_x_train.txt", header=None,  delim_whitespace=True)
total_acc_y_train = pd.read_csv("body_gyro_y_train.txt", header=None,  delim_whitespace=True)
total_acc_z_train = pd.read_csv("body_gyro_z_train.txt", header=None,  delim_whitespace=True)

body_acc_x_test  = pd.read_csv("body_acc_x_test.txt", header=None,  delim_whitespace=True)
body_acc_y_test  = pd.read_csv("body_acc_y_test.txt", header=None,  delim_whitespace=True)
body_acc_z_test  = pd.read_csv("body_acc_z_test.txt", header=None,  delim_whitespace=True)
body_gyro_x_test = pd.read_csv("body_gyro_x_test.txt", header=None,  delim_whitespace=True)
body_gyro_y_test = pd.read_csv("body_gyro_y_test.txt", header=None,  delim_whitespace=True)
body_gyro_z_test = pd.read_csv("body_gyro_z_test.txt", header=None,  delim_whitespace=True)
total_acc_x_test = pd.read_csv("body_gyro_x_test.txt", header=None,  delim_whitespace=True)
total_acc_y_test = pd.read_csv("body_gyro_y_test.txt", header=None,  delim_whitespace=True)
total_acc_z_test = pd.read_csv("body_gyro_z_test.txt", header=None,  delim_whitespace=True)

In [ ]:
feature = features
feature.drop(0,inplace=True, axis=1)

In [ ]:
#inseriamo l'intestazione al dataset
lista=[]
feat_transpa = feature.transpose()
for i in range(561):
    lista.append(feat_transpa.iloc[0][i])
X_test.columns=lista    
X_train.columns=lista

In [ ]:
# eliminiamo le colonne che contengono la stima mad in quanto quasi uguale a dev.std

stringa="mad()"
for col in X_train.columns:
    if(stringa in col):
        X_train.drop(labels=col, axis=1, inplace=True)
for col in X_test.columns:
    if(stringa in col):
        X_test.drop(labels=col, axis=1, inplace=True)
#for col in features:
#    if (stringa in col):
#        features.drop(labels=col,axis=1,inplace=True)

# ADVANCED CLUSTERING

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

## Gaussian Mixture

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gm1 = GaussianMixture(n_components=10, random_state=0, covariance_type='full', n_init=5)
gm1.fit(X_train)

y_pred_gm1 = gm1.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred_gm1))
print('F1-score %s' % f1_score(y_test, y_pred_gm1, average=None))

In [ ]:
gm1.means_

In [ ]:
gm1.covariances_

In [ ]:
gm1.precisions_

### How many K cluster?

### BIC criterion

In [ ]:
# BIC for GMM
#%%time
n_components = range(1, 20)
covariance_type = ['spherical', 'tied', 'diag', 'full']
score=[]
for cov in covariance_type:
    for n_comp in n_components:
        gmm=GaussianMixture(n_components=n_comp,covariance_type=cov)
        gmm.fit(X_train)
        score.append((cov,n_comp,gmm.bic(X_train)))
score

## X-Means

In [ ]:
#!pip install pyclustering

In [ ]:
# Prepare initial centers - amount of initial centers defines amount of clusters from which X-Means will start analysis.
#Pyclustering utilizza come metrica di distanza di default la euclidean square distance
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster.xmeans import xmeans, splitting_type
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import FCPS_SAMPLES

In [ ]:
xmeans_instance = xmeans(sample, initial_centers, 1000, random_state=random_seed )
xmeans_instance.process()
clusters = xmeans_instance.get_clusters()
centers = xmeans_instance.get_centers()

In [ ]:
len(clusters)

## New X_Means

In [ ]:
# X_means per scegliere il numero K di cluster. 
from pyclustering.cluster.silhouette import silhouette

random_seed = 1000
array=X_train.values
max_clust= 70
amount_initial_centers = 2
sample=array
initial_centers = kmeans_plusplus_initializer(sample, amount_initial_centers).initialize()
error=[]
sil=[]

for i in range(2,max_clust):
    xmeans_instance = xmeans(sample, initial_centers, i, random_state=random_seed )
    xmeans_instance.process()
    clusters = xmeans_instance.get_clusters()
    centers = xmeans_instance.get_centers()
    error.append(xmeans_instance.get_total_wce())
    silu = silhouette(sample, clusters).process().get_score()
    silu=np.array(silu)
    silu=silu.mean()
    sil.append(silu)

In [ ]:
for i in range(len(error)):
    print(i, "SSE: ", error[i], "Silhoeutte ", sil[i])

In [ ]:
y1=error
y2=sil
a=range(2,70)
color1="red"
color2="blue"

fig, ax1 = plt.subplots(figsize=(9,6))

ax1.set_xlabel('Number of cluster')
ax1.set_ylabel('SSE', color=color1)
ax1.plot(a, y1, color=color1)
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()  
ax2.set_ylabel('Silhouette', color=color2)  
ax2.plot(a, y2, color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

fig.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import Normalizer
from pyclustering.cluster.silhouette import silhouette

array=X_train.values
#X_train_norm = Normalizer(norm='l2').fit(array)
#X_train_normalized = X_train_norm.transform(array)

max_clust= 17
amount_initial_centers = 17
sample=array
initial_centers = kmeans_plusplus_initializer(sample, amount_initial_centers).initialize()
error_norm=[]
sil=[]

xmeans_instance = xmeans(sample, initial_centers)
xmeans_instance.process()
clusters = xmeans_instance.get_clusters()
centers = xmeans_instance.get_centers()
error_norm.append(xmeans_instance.get_total_wce())
silu = silhouette(sample, clusters).process().get_score()
silu=np.array(silu)
silu=silu.mean()
sil.append(silu)

In [ ]:
#distribuzione cluster
for i in range(len(clusters)):
    print(i,  len(clusters[i]))

In [ ]:
print("SSE",error_norm, "Numero di cluster", len(clusters), "silhouette:", sil )

In [ ]:
#PLOT CLUSTER
plt.figure(figsize=[10,6])
i=3
j=4
for indexes in clusters:
    plt.scatter(X_train.values[indexes,i], X_train.values[indexes,j], alpha=0.4)
for c in centers:
    plt.scatter(c[i], c[j], s=100, edgecolors='k')

## K_Means

In [ ]:
from pyclustering.cluster.kmeans import kmeans, kmeans_visualizer
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer


random_seed = 1000
sample=X_train.values
amount_initial_center=range(70)
error=[]
sil=[]

for i in range(2, len(amount_initial_center)):
    
    initial_centers = kmeans_plusplus_initializer(sample, amount_centers=i, random_state=random_seed).initialize()
    kmeans_instance = kmeans(sample, initial_centers)
    kmeans_instance.process()
    clusters = kmeans_instance.get_clusters()
    centers = kmeans_instance.get_centers()
    error.append(kmeans_instance.get_total_wce())
    silu = silhouette(sample, clusters).process().get_score()
    silu=np.array(silu)
    silu=silu.mean()
    sil.append(silu)

In [ ]:
for i in range(len(error)):
    print(i, "SSE: ", error[i], "Silhoeutte ", sil[i])

In [ ]:
y1=error
y2=sil
a=range(2,70)
color1="red"
color2="blue"

fig, ax1 = plt.subplots(figsize=(9,6))

ax1.set_xlabel('Number of cluster')
ax1.set_ylabel('SSE', color=color1)
ax1.plot(a, y1, color=color1)
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()  
ax2.set_ylabel('Silhouette', color=color2)  
ax2.plot(a, y2, color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

fig.tight_layout()
plt.show()

## OPTICS

In [ ]:
from sklearn.cluster import OPTICS
from sklearn.metrics import silhouette_score
from sklearn import metrics
#from skelarn.metrics import metrics.silhouette_score

In [ ]:
optics = OPTICS(min_samples=(528), max_eps=np.inf)
optics.fit(X_train)

In [ ]:
np.unique(optics.labels_, return_counts=True)

In [ ]:
#silhouette score senza considerare le labels=-1 in quanto sono noise points
sil=silhouette_score(X_train[optics.labels_ != -1], optics.labels_[optics.labels_ != -1])
sil

In [ ]:
#PLOT
for cluster_id in np.unique(optics.labels_)[:20]:
    indexes = np.where(optics.labels_==cluster_id)
    plt.scatter(X_train.values[indexes,i], X_train.values[indexes,j], alpha=0.4)

In [ ]:
#da questo grafico si potrebbe capire dove c'è il salto della reach_dist che probabilmente c'è un cluster.
#più l'altezza dei bin è bassa, più concentrato e denso è il cluster
plt.hist(optics.reachability_[1:])
plt.show()

In [ ]:
optics1 = OPTICS(min_samples=5, max_eps=np.inf, cluster_method='dbscan', eps=0.3)
optics1.fit(X_train)

In [ ]:
#Cluster method = 'XI', scegliamo il miglior numero di min_samples in base alla silhouette
plt.figure(figsize = (9,6))
min_samples_to_test = [2,3,4,5,6,7,8,9,10,11,12,13,14,15] #massimo 20 perchè poi vede solo noise points
values = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]

#range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
plt.rcParams['axes.facecolor'] = 'w'
silhouette_avg = []
error =[]
for min_samples in min_samples_to_test:
    optics1 = OPTICS(min_samples=min_samples, max_eps=np.inf, cluster_method='xi')
    optics1.fit(X_train)
    cluster_labels = optics1.labels_
    silhouette_avg.append(silhouette_score(X_train.values[optics1.labels_ != -1], optics1.labels_[optics1.labels_ != -1]))
    
plt.plot(min_samples_to_test, silhouette_avg, marker='.', markersize=10, markerfacecolor='black')
plt.xlabel('\n Min_samples', fontsize = 15, fontweight = 'bold')
plt.ylabel('Silhouette score', fontsize = 15, fontweight = 'bold')
plt.grid(True)
plt.xticks(values)
plt.show()

In [ ]:
silhouette_avg

In [ ]:
optics1 = OPTICS(min_samples=11, max_eps=np.inf, cluster_method='xi')
optics1.fit(X_train)

In [ ]:
np.unique(optics1.labels_, return_counts=True)

In [ ]:
plt.figure(figsize=(9,6))
for cluster_id in np.unique(optics1.labels_)[:20]:
    indexes = np.where(optics1.labels_==cluster_id)
    plt.scatter(X_train.values[indexes,3], X_train.values[indexes,4], alpha=0.4)

# Transactional Clustering

## K-Mode

In [ ]:
#!pip install kmodes

In [ ]:
#!pip install plotnine

In [ ]:
# Import module for data visualization
from plotnine import *
import plotnine
# Data visualization with matplotlib
import matplotlib.pyplot as plt
# Use the theme of ggplot
plt.style.use('ggplot')
# Import module for k-protoype cluster
from kmodes.kmodes import KModes

In [ ]:
X_cat = list()
for i in range(len(X_train)):
    X_cat.append(pd.qcut(X_train.values[i], q=[0.0, 0.25, 0.5, 0.75, 1.0], labels=False))
X_cat = np.array(X_cat)

In [ ]:
X_cat_test = list()
for i in range(len(X_test)):
    X_cat_test.append(pd.qcut(X_test.values[i], q=[0.0, 0.25, 0.5, 0.75, 1.0], labels=False))
X_cat_test = np.array(X_cat_test)

In [ ]:
X_cat_test=pd.DataFrame(data=X_cat_test)
X_cat_test

In [ ]:
X_cat=pd.DataFrame(data=X_cat)
X_cat

In [ ]:
np.unique(X_cat_test, return_counts=True)

In [ ]:
cost = []
costo=[]
for cluster in range(2, 20):    
    km = KModes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
    km = km.fit(X_cat)
    costo.append(km.cost_)
    cost=costo[0]
    

In [ ]:
#Elbow Method to find optimal number of cluster
df_cost = pd.DataFrame({'Cluster': range(2, 20), 'Cost': costo})
# Data viz
plotnine.options.figure_size = (7, 4.5)
(
    ggplot(data = df_cost)+
    geom_line(aes(x = 'Cluster',
                  y = 'Cost'))+
    geom_point(aes(x = 'Cluster',
                   y = 'Cost'))+
    geom_label(aes(x = 'Cluster',
                   y = 'Cost',
                   label = 'Cluster'),
               size = 8,
               nudge_y = 1000) +
    labs(title = 'Optimal number of cluster with Elbow Method')+
    xlab('Number of Clusters k')+
    ylab('Cost')+
    theme_minimal()
)

In [ ]:
costo

In [ ]:
km = KModes(n_jobs = -1, n_clusters = 7, init = 'Huang', random_state = 0)
km = km.fit(X_cat)

In [ ]:
km.cluster_centroids_

In [ ]:
np.unique(km.labels_, return_counts=True)

In [ ]:
#ditribuzione oggetti cluster con le classi del dataset
pclass_xt = pd.crosstab(km.labels_, y_train[0])
pclass_xt

## To see characteristics into a single cluster

In [ ]:
cluster2=np.where(km.labels_ == 2)
y_train2=y_train.iloc[cluster2][:]
X_cat2=X_cat.iloc[cluster2][:]

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score 
X_cat2_new, X_test2_new, y_train2_new, y_test2_new = train_test_split(X_cat2, y_train2, test_size = 0.33, random_state = 42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

clf_dt = DecisionTreeClassifier(criterion='gini', max_depth= None, random_state=42)
clf_dt.fit(X_cat2_new, y_train2_new) #df3

In [ ]:
len(X_cat2)

In [ ]:
y_pred=clf_dt.predict(X_test2_new)
print('Accuracy %s' % accuracy_score(y_test2_new, y_pred))
print('F1-score %s' % f1_score(y_test2_new, y_pred, average=None))
print(classification_report(y_test2_new, y_pred))

In [ ]:
clf_dt.feature_importances_
df=pd.DataFrame({'Features_names': X_train.columns, 'Importances': clf_dt.feature_importances_})
df

In [ ]:
df1= df.sort_values(by='Importances', ascending=False)
df2=df1[:6]
df2

In [ ]:
#1 WALKING #2 WALKING_UPSTAIRS #3 WALKING_DOWNSTAIRS #4 SITTING #5 STANDING #6 LAYING
print(np.unique(X_cat2[198]), np.unique( y_train2[0], return_counts=True))
# si vede che per questa var sono presenti solo valori alti ed in particolare identifica 800 record Walking downstairs

In [ ]:
#in teoria andando a plottare i cluster per la variabile tBodyAccMag-std() discretizzata si dovrebbe vedere che
#effettivamente per un valore alto ovvero 3 si identifica un gruppo di 
pclass_xt2 = pd.crosstab(X_cat2[198], y_train2[0])
pclass_xt2

## ROCK

In [ ]:
#NON usato
from pyclustering.cluster import rock

In [ ]:
rc = rock.rock(X_cat[:100], eps=0.7, number_clusters=4, threshold=0.5)
rc.process()

In [ ]:
clusters_rc = rc.get_clusters()

In [ ]:
clusters_rc

In [ ]:
for i, indexes in enumerate(clusters):
    print(i, X_cat[indexes[:5]])
    #plt.scatter(X_train.values[indexes,1], X_train.values[indexes,2], alpha=0.4)

## Transactional Clustering TX-Means

In [2]:
# https://github.com/riccotti/TX-Means